<a href="https://colab.research.google.com/github/kridtapon/SAR-TrendLock-System/blob/main/SAR_TrendLock_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Parabolic SAR manually
def calculate_parabolic_sar(df, initial_af=0.02, max_af=0.2):
    """
    Calculate the Parabolic SAR (Stop and Reverse) manually.
    """
    sar = [df['Close'][0]]  # Initial SAR value is set to the first closing price
    af = initial_af  # Acceleration Factor (AF)
    ep = df['High'][0]  # Extreme Point (EP) for an uptrend, starts as the first high value
    uptrend = True  # We start with an uptrend
    for i in range(1, len(df)):
        if uptrend:
            sar.append(sar[-1] + af * (ep - sar[-1]))
            # Ensure SAR does not exceed the previous low
            sar[-1] = min(sar[-1], df['Low'][i-1])
            if df['Low'][i] < sar[-1]:
                uptrend = False
                sar[-1] = ep  # Set SAR to EP when trend reverses
                ep = df['Low'][i]  # New EP for downtrend
                af = initial_af  # Reset AF
        else:
            sar.append(sar[-1] + af * (ep - sar[-1]))
            # Ensure SAR does not exceed the previous high
            sar[-1] = max(sar[-1], df['High'][i-1])
            if df['High'][i] > sar[-1]:
                uptrend = True
                sar[-1] = ep  # Set SAR to EP when trend reverses
                ep = df['High'][i]  # New EP for uptrend
                af = initial_af  # Reset AF

        # Update the AF based on the maximum value
        if (ep == df['High'][i] and uptrend) or (ep == df['Low'][i] and not uptrend):
            af = min(af + initial_af, max_af)  # Increment AF but limit it to max_af
    return pd.Series(sar, index=df.index)

# Define the stock symbol and time period
symbol = 'META'  # SPY is the symbol for the S&P 500 ETF
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate the Parabolic SAR
df['SAR'] = calculate_parabolic_sar(df)

# Entry Condition: Price above the SAR and SAR dots switch from above to below the price
df['Entry'] = (df['Close'] > df['SAR']) & (df['SAR'] < df['Close'].shift(1))

# Exit Condition: Price below the SAR and SAR dots switch from below to above the price
df['Exit'] = (df['Close'] < df['SAR']) & (df['SAR'] > df['Close'].shift(1))

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-2-1b4a5bdd30ea>:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sar = [df['Close'][0]]  # Initial SAR value is set to the first closing price
<ipython-input-2-1b4a5bdd30ea>:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ep = df['High'][0]  # Extreme Point (EP) for an uptrend, starts as the first high value
<ipython-input-2-1b4a5bdd30ea>:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a

Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           197709.875305
Total Return [%]                        97.709875
Benchmark Return [%]                   180.172876
Max Gross Exposure [%]                      100.0
Total Fees Paid                         980.21986
Max Drawdown [%]                        71.699097
Max Drawdown Duration                       762.0
Total Trades                                    8
Total Closed Trades                             7
Total Open Trades                               1
Open Trade PnL                      146150.910093
Win Rate [%]                                  0.0
Best Trade [%]                          -3.602992
Worst Trade [%]                        -25.310716
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                    -8.741292


In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Parabolic SAR manually
def calculate_parabolic_sar(df, initial_af=0.02, max_af=0.2):
    """
    Calculate the Parabolic SAR (Stop and Reverse) manually.
    """
    sar = [df['Close'][0]]  # Initial SAR value is set to the first closing price
    af = initial_af  # Acceleration Factor (AF)
    ep = df['High'][0]  # Extreme Point (EP) for an uptrend, starts as the first high value
    uptrend = True  # We start with an uptrend
    for i in range(1, len(df)):
        if uptrend:
            sar.append(sar[-1] + af * (ep - sar[-1]))
            # Ensure SAR does not exceed the previous low
            sar[-1] = min(sar[-1], df['Low'][i-1])
            if df['Low'][i] < sar[-1]:
                uptrend = False
                sar[-1] = ep  # Set SAR to EP when trend reverses
                ep = df['Low'][i]  # New EP for downtrend
                af = initial_af  # Reset AF
        else:
            sar.append(sar[-1] + af * (ep - sar[-1]))
            # Ensure SAR does not exceed the previous high
            sar[-1] = max(sar[-1], df['High'][i-1])
            if df['High'][i] > sar[-1]:
                uptrend = True
                sar[-1] = ep  # Set SAR to EP when trend reverses
                ep = df['High'][i]  # New EP for uptrend
                af = initial_af  # Reset AF

        # Update the AF based on the maximum value
        if (ep == df['High'][i] and uptrend) or (ep == df['Low'][i] and not uptrend):
            af = min(af + initial_af, max_af)  # Increment AF but limit it to max_af
    return pd.Series(sar, index=df.index)

# Calculate the Average True Range (ATR)
def calculate_atr(df, window=14):
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift(1))
    low_close = np.abs(df['Low'] - df['Close'].shift(1))
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window).mean()
    return atr

# Define the stock symbol and time period
symbol = 'META'  # SPY is the symbol for the S&P 500 ETF
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate the Parabolic SAR
df['SAR'] = calculate_parabolic_sar(df)

# Calculate the ATR
df['ATR'] = calculate_atr(df)

# Define an ATR threshold to filter sideways market (you can adjust this value)
atr_threshold = df['ATR'].quantile(0.5)  # Example: bottom 25% of ATR values

# Entry Condition: Price above the SAR and SAR dots switch from above to below the price
# Only enter if ATR is above the threshold
df['Entry'] = (df['Close'] > df['SAR']) & (df['SAR'] < df['Close'].shift(1)) & (df['ATR'] > atr_threshold)

# Exit Condition: Price below the SAR and SAR dots switch from below to above the price
df['Exit'] = (df['Close'] < df['SAR']) & (df['SAR'] > df['Close'].shift(1)) & (df['ATR'] > atr_threshold)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-6-0da2680883c1>:11: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-6-0da2680883c1>:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-6-0da2680883c1>:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-6-0da2680883c1>:20: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future 

Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           158880.243854
Total Return [%]                        58.880244
Benchmark Return [%]                   180.172876
Max Gross Exposure [%]                      100.0
Total Fees Paid                        560.117356
Max Drawdown [%]                        79.653591
Max Drawdown Duration                       834.0
Total Trades                                    4
Total Closed Trades                             3
Total Open Trades                               1
Open Trade PnL                      108406.849026
Win Rate [%]                                  0.0
Best Trade [%]                          -5.742175
Worst Trade [%]                        -42.136696
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                   -18.472222
